In [1]:
import pandas as pd
file_path = 'Case Study Data - Read Only.xlsx'
df = pd.ExcelFile(file_path)
df = df.parse('Sheet1')

In [5]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

df['DATE'] = pd.to_datetime(df['DATE'])
df['Month-Year'] = df['DATE'].dt.strftime('%Y-%m')
df['Value'] = df['QUANTITY'] * df['UNIT PRICE']

monthly_sales = df.groupby('Month-Year').agg({'QUANTITY': 'sum', 'Value': 'sum'}).reset_index()
category_sales = df.groupby('ANONYMIZED CATEGORY').agg({'Value': 'sum'}).reset_index()
top_products_quantity = df.groupby('ANONYMIZED PRODUCT').agg({'QUANTITY': 'sum'}).nlargest(5, 'QUANTITY').reset_index()
top_products_value = df.groupby('ANONYMIZED PRODUCT').agg({'Value': 'sum'}).nlargest(5, 'Value').reset_index()

business_metrics = df.groupby('ANONYMIZED BUSINESS').agg(
    Total_Quantity=('QUANTITY', 'sum'),
    Total_Value=('Value', 'sum'),
    Frequency=('DATE', 'count')
).reset_index()
business_metrics['Segment'] = pd.qcut(business_metrics['Total_Value'], q=3, labels=['Low Value', 'Medium Value', 'High Value'])
segment_distribution = business_metrics['Segment'].value_counts().reset_index()
segment_distribution.columns = ['Segment', 'Count']

monthly_sales['Quantity_zscore'] = (monthly_sales['QUANTITY'] - monthly_sales['QUANTITY'].mean()) / monthly_sales['QUANTITY'].std()
monthly_sales['Value_zscore'] = (monthly_sales['Value'] - monthly_sales['Value'].mean()) / monthly_sales['Value'].std()
anomalies = monthly_sales[(abs(monthly_sales['Quantity_zscore']) > 3) | (abs(monthly_sales['Value_zscore']) > 3)]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Sales Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        dcc.Graph(
            id='sales-by-category',
            figure=px.bar(category_sales, x='ANONYMIZED CATEGORY', y='Value', title='Total Sales by Category')
        ),
        dcc.Graph(
            id='sales-trends',
            figure=px.line(monthly_sales, x='Month-Year', y='Value', title='Sales Trends Over Time')
        )
    ], style={'display': 'flex'}), 
    
    html.Div([
        dcc.Graph(
            id='top-products-quantity',
            figure=px.bar(top_products_quantity, x='ANONYMIZED PRODUCT', y='QUANTITY', title='Top 5 Products by Quantity')
        ),
        dcc.Graph(
            id='top-products-value',
            figure=px.bar(top_products_value, x='ANONYMIZED PRODUCT', y='Value', title='Top 5 Products by Value')
        )
    ], style={'display': 'flex'}),
    
    html.Div([
        dcc.Graph(
            id='customer-segmentation',
            figure=px.pie(segment_distribution, values='Count', names='Segment', title='Customer Segmentation')
        ),
        dcc.Graph(
            id='anomalies',
            figure=px.scatter(anomalies, x='Month-Year', y='Value', title='Anomalies in Sales Data')
        )
    ], style={'display': 'flex'})
])

app.run_server(mode='inline')

C:\Users\Amolo Washington\anaconda3\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

